In [ ]:
# library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from pydap.net import create_session
from pydap.client import open_url
import pydap

# local imports
from helperLibrary import subset, crop, profileLocationPlot


In [ ]:
profilesETP = xr.open_dataset(
    "../data/ARGO_VortexProfiles/subsetProfiles/EasternTropicalPacific.nc"
)

In [ ]:
import cf_xarray

In [ ]:
import gsw_xarray as gsw_xr

In [ ]:
pres = gsw_xr.p_from_z(
    -profilesETP['z'],
    profilesETP['latitude'],
)

In [ ]:
uvel_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/uvel'
vvel_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/vvel'
theta_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/theta'
salt_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/salt'
ECCO2_uvel = xr.open_dataset(
    uvel_url,
    engine='pydap'
)
ECCO2_vvel = xr.open_dataset(
    vvel_url,
    engine='pydap'
)
ECCO2_theta = xr.open_dataset(
    theta_url,
    engine='pydap'
)
ECCO2_salt = xr.open_dataset(
    salt_url,
    engine='pydap'
)

In [ ]:
ECCO2_uvel = ECCO2_uvel.sel(
    lat=slice(5, 23.75),
    lon=slice(245, 267.5),
    time=slice(np.datetime64('2004-01-01'), np.datetime64('2022-12-31')),
    drop=True
)

In [ ]:
ECCO2_vvel = ECCO2_vvel.sel(
    lat=slice(5, 23.75),
    lon=slice(245, 267.5),
    time=slice(np.datetime64('2004-01-01'), np.datetime64('2022-12-31')),
    drop=True
)

In [ ]:
ECCO2velocities = xr.merge(
    [ECCO2_uvel, ECCO2_vvel]
)

In [ ]:
ECCO2velocities

In [ ]:
ECCO2velocities = ECCO2velocities.where(
    (ECCO2velocities['lat'] <= 18) |
    (ECCO2velocities['lon'] <= 240), drop=True
)

In [ ]:
climatology = xr.open_dataset(
    "../data/ARGO_RG2019_Climatology/subsetClimatology/RG_ArgoClim_Merged_ETP.nc"
)

In [ ]:
climatology